In [3]:
import torch
from botorch.models import SingleTaskGP
from botorch.sampling.pathwise.posterior_samplers import draw_matheron_paths
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.likelihoods import GaussianLikelihood

# Example data
train_x = torch.rand(10, 1)  # Feature vector of 10 points
train_y = torch.sin(train_x).squeeze()  # Target vector

# Define GP model
likelihood = GaussianLikelihood()
model = SingleTaskGP(train_x, train_y.unsqueeze(-1), likelihood)

# Model training (for completeness)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
# Here you would optimize the MLL using your choice of optimizer...

# Prepare for sampling
model.eval()  # Put the model in evaluation mode

# Sampling parameters
sample_shape = torch.Size([10])  # For example, draw 10 samples

# Draw samples
samples = draw_matheron_paths(model=model, sample_shape=sample_shape, X=train_x)


AttributeError: 'GaussianLikelihood' object has no attribute 'dim'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming `samples` is the output from draw_matheron_paths
# And `train_x`, `train_y` are your training data

# Generate test points for prediction (for visualization)
test_x = torch.linspace(0, 1, 100)  # Adjust the range and density as needed

# Get model's prediction (mean and variance) at test points
model.eval()  # Make sure the model is in evaluation mode
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = model(test_x.unsqueeze(-1))

# Extract mean and variance
mean = observed_pred.mean.numpy()
stddev = observed_pred.stddev.numpy()

# Plotting
plt.figure(figsize=(10, 6))

# Plot mean and confidence interval
plt.plot(test_x, mean, 'k', lw=2, label='Mean prediction')
plt.fill_between(test_x, mean - 1.96*stddev, mean + 1.96*stddev, alpha=0.5, label='95% confidence interval')

# Plot samples
for i in range(samples.shape[0]):  # Assuming the first dimension of samples is the number of samples
    plt.plot(train_x.numpy(), samples[i].numpy(), 'r-', lw=1, alpha=0.3)

# Scatter plot of training data
plt.scatter(train_x, train_y, c='b', s=50, zorder=3, label='Training data')

plt.legend()
plt.title('GP Samples with Mean Prediction and Confidence Interval')
plt.xlabel('Input')
plt.ylabel('Output')
plt.show()
